In [1]:
# Import necessary libraries
import openai
from openai.embeddings_utils import get_embedding, cosine_similarity
import pandas as pd
import numpy as np

In [3]:
# Load the csv file as dataframe.
df = pd.read_csv('./data/face_cosmetics_products_data.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Product_ID        36 non-null     int64 
 1   Product_Category  36 non-null     object
 2   Product_Type      36 non-null     object
 3   Brand_Name        36 non-null     object
 4   Product_Name      36 non-null     object
 5   Product_Details   36 non-null     object
 6   Benefits          36 non-null     object
dtypes: int64(1), object(6)
memory usage: 2.1+ KB


In [5]:
# Configure the baseline configuration of the OpenAI library.
openai.api_type = "azure"
openai.api_base = "https://PLESAE_ENTER_YOUR_OWNED_AOAI_RESOURCE_NAME.openai.azure.com/"
openai.api_version = "2022-12-01"
openai.api_key = "PLEASE_ENTER_YOUR_OWNED_AOAI_SERVICE_KEY"
engine = "PLEASE_ENTER_YOUR_OWNED_AOAI_TEXT_EMBEDDING_MODEL_NAME"

In [6]:
# Perform word embedding for each product name in the dataframe in vector form.
embeddings = [get_embedding(text, engine=engine) for text in df['Product_Details'].values]

In [7]:
# Calculate the cosine similarity between each product name.
similarity_matrix = np.zeros((len(embeddings), len(embeddings)))
for i in range(len(embeddings)):
    for j in range(len(embeddings)):
        if i == j:
            similarity_matrix[i,j] = 1.0
        elif i < j:
            similarity_matrix[i,j] = cosine_similarity(embeddings[i], embeddings[j])
        else:
            similarity_matrix[i,j] = similarity_matrix[j,i]

In [8]:
# Save the similarity matrix to a CSV file.
np.savetxt('./data/similarity_matrix_fc_d.csv', similarity_matrix, delimiter=',')

In [8]:
# Reload the similarity matrix from the CSV file.
# similarity_matrix = np.loadtxt('./data/similarity_matrix_fc_d.csv', delimiter=',')

In [9]:
# Create a list of Product_Name.
p_name = pd.Series(df['Product_Name'])
p_name

0     FACEFINITY ALL DAY FLAWLESS 3 IN 1 VEGAN FOUND...
1                MIRACLE PURE SKIN IMPROVING FOUNDATION
2                         FACEFINITY COMPACT FOUNDATION
3                              MIRACLE TOUCH FOUNDATION
4                  MAX FACTOR CRÈME PUFF POWDER COMPACT
5                 FACEFINITY ALL DAY FLAWLESS CONCEALER
6                                MIRACLE PURE CONCEALER
7                                RADIANT LIFT CONCEALER
8                                 MASTERTOUCH CONCEALER
9                            MIRACLE CONTOURING PALETTE
10                                     MIRACLE GLOW DUO
11                             MIRACLE PURE CREAM BLUSH
12                           MIRACLE TOUCH CREAMY BLUSH
13       MIRACLE PURE VITAMIN C SKIN-ILLUMINATING SERUM
14                           BROW SHAPER EYEBROW PENCIL
15                                     BROW REVIVAL GEL
16                        BROWFINITY LONGWEAR BROW TINT
17                                       EYEBROW

In [10]:
# Create function for Product Recommendations.
def product_recommendations(Product_Name, cos_sim = similarity_matrix):
    recommended_product = []
    idx = p_name[p_name == Product_Name].index[0]
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)
    top_10_indexes = list(score_series.iloc[1:11].index)
    for i in top_10_indexes:
        recommended_product.append(list(df['Product_Name'])[i])
    return recommended_product

In [11]:
# Perform Product Recommendations. It is based on Product Name to figure out the similarity.
product_recommendations('MIRACLE PURE SKIN IMPROVING FOUNDATION')

['MIRACLE PURE CONCEALER',
 'MIRACLE PURE CREAM BLUSH',
 'MIRACLE TOUCH FOUNDATION',
 'MIRACLE PURE VITAMIN C SKIN-ILLUMINATING SERUM',
 'FACEFINITY ALL DAY FLAWLESS 3 IN 1 VEGAN FOUNDATION',
 'FACEFINITY ALL DAY FLAWLESS CONCEALER',
 'FACEFINITY COMPACT FOUNDATION',
 'FALSE LASH EFFECT LASH & BROW SERUM',
 'MIRACLE CONTOURING PALETTE',
 'MAX FACTOR CRÈME PUFF POWDER COMPACT']

In [12]:
df_product_sim_matrix = pd.DataFrame(similarity_matrix, index = p_name, columns = p_name)
df_product_sim_matrix

Product_Name,FACEFINITY ALL DAY FLAWLESS 3 IN 1 VEGAN FOUNDATION,MIRACLE PURE SKIN IMPROVING FOUNDATION,FACEFINITY COMPACT FOUNDATION,MIRACLE TOUCH FOUNDATION,MAX FACTOR CRÈME PUFF POWDER COMPACT,FACEFINITY ALL DAY FLAWLESS CONCEALER,MIRACLE PURE CONCEALER,RADIANT LIFT CONCEALER,MASTERTOUCH CONCEALER,MIRACLE CONTOURING PALETTE,...,FALSE LASH EPIC MASCARA,LASH REVIVAL EYELASH PRIMER,VOLUME INFUSION MASCARA,FALSE LASH EFFECT XXL MASCARA,VELVET MATTES LIPSTICK,COLOUR ELIXIR LIPSTICK,PRIYANKA COLOUR ELIXIR LIPSTICK,LIPFINITY LIQUID LIPSTICK,COLOUR ELIXIR LIP LINER,HONEY LACQUER
Product_Name,,,,,,,,,,,,,,,,,,,,,
FACEFINITY ALL DAY FLAWLESS 3 IN 1 VEGAN FOUNDATION,1.000000,0.870842,0.890601,0.865370,0.851012,0.942373,0.867197,0.824847,0.840727,0.825347,...,0.786554,0.788451,0.776204,0.825455,0.823918,0.822311,0.830234,0.847968,0.800844,0.807208
MIRACLE PURE SKIN IMPROVING FOUNDATION,0.870842,1.000000,0.854360,0.903633,0.848378,0.859045,0.930945,0.842222,0.817671,0.850289,...,0.807083,0.812949,0.813931,0.840582,0.816759,0.831308,0.837129,0.828171,0.804282,0.811468
FACEFINITY COMPACT FOUNDATION,0.890601,0.854360,1.000000,0.868364,0.854979,0.873910,0.840614,0.809870,0.835103,0.828077,...,0.773694,0.774042,0.778835,0.792518,0.813354,0.792395,0.801748,0.836785,0.791915,0.793225
MIRACLE TOUCH FOUNDATION,0.865370,0.903633,0.868364,1.000000,0.838655,0.862983,0.876257,0.859108,0.863521,0.855525,...,0.800465,0.803983,0.808155,0.809364,0.830532,0.827308,0.810810,0.834674,0.792302,0.819779
MAX FACTOR CRÈME PUFF POWDER COMPACT,0.851012,0.848378,0.854979,0.838655,1.000000,0.846000,0.843831,0.793751,0.830156,0.853544,...,0.794534,0.776394,0.770992,0.830457,0.822953,0.810840,0.841607,0.817066,0.782871,0.798628
FACEFINITY ALL DAY FLAWLESS CONCEALER,0.942373,0.859045,0.873910,0.862983,0.846000,1.000000,0.897708,0.875015,0.877276,0.840307,...,0.787833,0.765901,0.779242,0.812467,0.823563,0.819457,0.827236,0.846822,0.795653,0.800492
MIRACLE PURE CONCEALER,0.867197,0.930945,0.840614,0.876257,0.843831,0.897708,1.000000,0.873319,0.870097,0.874360,...,0.783786,0.784841,0.791638,0.812662,0.799349,0.812474,0.823418,0.797174,0.783432,0.806961
RADIANT LIFT CONCEALER,0.824847,0.842222,0.809870,0.859108,0.793751,0.875015,0.873319,1.000000,0.863317,0.818395,...,0.801637,0.792898,0.796664,0.787039,0.780547,0.802108,0.789664,0.805620,0.791883,0.796935
MASTERTOUCH CONCEALER,0.840727,0.817671,0.835103,0.863521,0.830156,0.877276,0.870097,0.863317,1.000000,0.850945,...,0.790324,0.783310,0.793196,0.805875,0.825771,0.805456,0.804645,0.809005,0.798086,0.800368
